<a href="https://colab.research.google.com/github/hayunjong83/computer_vision_implement_research/blob/master/keras_official_examples/simple_mnist_convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple MNIST convnet
---

원본 글 : [Simple MNIST convnet](https://keras.io/examples/vision/mnist_convnet/)

이미 여러 서적 및 블로그를 통해 소개된 MNIST를 분류하는
 간단한 컨볼루션 신경망의 예이다.

In [1]:
import numpy as np
from tensorflow import keras

## 데이터 준비
---
mnist 데이터셋을 로드하고 **정규화**를 수행해 픽셀값의 범위가 [0, 1] 이 되도록 한다.

**keras.utils.to_categorical()**함수을 활용하면, 레이블 값을 one-hot vector로 바꿔준다(**one-hot encoding**)

In [2]:
# Model / data parameters 
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape: ", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrics
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


11493376/11490434 [==============================] - 0s 0us/step
x_train shape:  (60000, 28, 28, 1)
60000 train samples
10000 test samples


input_shape를 단순히 이미지 크기가 아닌 한 개의 채널 수까지 포함한 (28, 28, 1)로 설정했으므로, **numpy.expand_dims()**를 사용해 마지막 차원을 추가해줬다.

## 모델 구성
---
각각 2개의 합성곱(convolution) 층과 최대 풀링(maxpooling) 층을 가지고, 마지막 클래수의 개수만큼 뉴런을 가진 완전연결층을 출력층으로 구성하였다.
- **[keras.layers.Conv2D()](https://keras.io/api/layers/convolution_layers/convolution2d/)**: convolution layer
- **[keras.layers.MaxPooling2D()](https://keras.io/api/layers/pooling_layers/max_pooling2d/)** : maxpooling layer
- **[keras.layers.Dense()](https://keras.io/api/layers/core_layers/dense/)** : completely connected layer

CNN에서는 보통 40% ~ 50%의 Dropout 비율을 가지는데, 여기서는 간단한 구조이므로 Dropout(0.5)를 적용하였다.

In [3]:
model = keras.Sequential(
    [
     keras.Input(shape = input_shape),
     keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
     keras.layers.MaxPooling2D(pool_size=(2,2)),
     keras.layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
     keras.layers.MaxPooling2D(pool_size=(2,2)),
     keras.layers.Flatten(),
     keras.layers.Dropout(0.5),
     keras.layers.Dense(num_classes, activation="softmax"),     
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

## 모델 훈련
---
배치 크기는 128개로 하고, 15번의 에포크 동안 훈련을 진행한다.

참고로 데이터의 레이블이 원-핫 인코딩 되지 않는 경우라면, 손실함수로 "sparse_categorical_crossentropy"를 사용하면 된다. 

In [4]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

**fit()**에서 따로 검증데이터를 지정하지 않고, **validation_split**을 설정하면, 지정된 비율만큼의 검증데이터를 훈련데이터의 끝에서 뽑아낸다.

[참고](https://keras.io/getting_started/faq/#in-fit-how-is-the-validation-split-computed)에 따르면, 검증데이터를 제외하고 훈련데이터를 섞게되므로(shuffle) 매 에포크마다 같은 검증데이터를 사용하게 된다.

마지막으로 validation_split을 사용하기 위해서는 입력 데이터는 numpy 배열이어야 한다.

In [5]:
model.fit(x_train, y_train, batch_size=batch_size, epochs = epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 4s 10ms/step - loss: 0.3778 - accuracy: 0.8850 - val_loss: 0.0907 - val_accuracy: 0.9748
Epoch 2/15
422/422 [==============================] - 4s 8ms/step - loss: 0.1154 - accuracy: 0.9652 - val_loss: 0.0592 - val_accuracy: 0.9833
Epoch 3/15
422/422 [==============================] - 3s 8ms/step - loss: 0.0880 - accuracy: 0.9735 - val_loss: 0.0475 - val_accuracy: 0.9877
Epoch 4/15
422/422 [==============================] - 3s 8ms/step - loss: 0.0728 - accuracy: 0.9781 - val_loss: 0.0415 - val_accuracy: 0.9905
Epoch 5/15
422/422 [==============================] - 3s 8ms/step - loss: 0.0620 - accuracy: 0.9811 - val_loss: 0.0385 - val_accuracy: 0.9908
Epoch 6/15
422/422 [==============================] - 3s 8ms/step - loss: 0.0561 - accuracy: 0.9827 - val_loss: 0.0356 - val_accuracy: 0.9905
Epoch 7/15
422/422 [==============================] - 3s 8ms/step - loss: 0.0508 - accuracy: 0.9844 - val_loss: 0.0333 - val_accuracy: 0.9903
Epoch

## 모델 평가
---
[evaluate()](https://keras.io/api/models/model_training_apis/#evaluate-method)에서 verbose 플래그를 통해 프로그레스바의 출력여부를 결정할 수 있다.

리턴값은 metrics이 지정되지 않은 경우는 손실값(loss)만 있고, 지정된 경우는 손실값, 메트릭 값...이 리스트로 반환된다. 따라서 여기에서 **score[0]**은 테스트 데이터의 *loss*이고, **score[1]**은 테스트 데이터의 *accuracy*다.

In [6]:
score = model.evaluate(x_test, y_test, verbose = 0)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

Test loss:  0.02356906048953533
Test accuracy:  0.9919999837875366
